In [19]:
import pandas as pd

# Знакомимся с новыми данными: коронавирус

In [20]:
covid_data = pd.read_csv('data/covid_data.csv')
display(covid_data.head())

,date,province/state,country,confirmed,deaths,recovered
0,01/22/2020,Anhui,China,1.0,0.0,0.0
1,01/22/2020,Beijing,China,14.0,0.0,0.0
2,01/22/2020,Chongqing,China,6.0,0.0,0.0
3,01/22/2020,Fujian,China,1.0,0.0,0.0
4,01/22/2020,Gansu,China,0.0,0.0,0.0


In [21]:
vaccinations_data = pd.read_csv('data/country_vaccinations.csv')
vaccinations_data = vaccinations_data[
    ['country', 'date', 'total_vaccinations', 
     'people_vaccinated', 'people_vaccinated_per_hundred',
     'people_fully_vaccinated', 'people_fully_vaccinated_per_hundred',
     'daily_vaccinations', 'vaccines']
]

## Немного об особенностях данных
Обратите внимание, что признаки confirmed, deaths, recovered, total_vaccination, people_vaccinated, people_fully_vaccinated — это суммарные показатели по стране, то есть с каждым днём они должны расти. Такие признаки называют накопительными.

Возникает большое желание объединить таблицы. Для этого необходимо учитывать следующие нюансы:

1 В таблице covid_data необходимо предварительно рассчитать суммарное ежедневное число заболевших во всех провинциях/штатах в каждой стране.

2 В таблицах не совпадает число стран, а иногда и их названия. 
При объединении таблиц по столбцу мы определённо теряем данные (в данной задаче потери незначительны). Избежать этого можно ручными преобразованиями данных — искать различия в названиях стран в таблицах и преобразовывать их. Однако это не является темой данного модуля.

3 Таблицы имеют разные периоды наблюдений (вакцины появились позже, чем сам вирус). Объединив данные с типом inner, мы можем потерять большое количество наблюдений в таблице covid_data.


## Предобработка данных

In [22]:
# Группируем таблицу по дате и названию страны и рассчитываем суммарные показатели по всем регионам
# Тем самым переходим от данных по регионам к данным по странам
covid_data = covid_data.groupby(
    ['date', 'country'], 
    as_index=False
)[['confirmed', 'deaths', 'recovered']].sum()

In [23]:
# Преобразуем даты в формат datetime с помощью функции pd.to_datetime
covid_data['date'] = pd.to_datetime(covid_data['date'])

In [24]:
# Создадим признак больных на данный момент (active)
# Для этого вычтем из общего числа зафиксированных случаев число смертей и число выздоровевших пациентов
covid_data['active'] = covid_data['confirmed'] - covid_data['deaths'] - covid_data['recovered']

In [25]:
# Создадим признак ежедневного прироста числа заболевших, умерших и выздоровевших людей
# Для этого отсортируем данные по названиям стран, а затем по датам
# После этого произведём группировку по странам и рассчитаем разницу между «вчера и сегодня» с помощью метода diff()
covid_data = covid_data.sort_values(by=['country', 'date'])
covid_data['daily_confirmed'] = covid_data.groupby('country')['confirmed'].diff()
covid_data['daily_deaths'] = covid_data.groupby('country')['deaths'].diff()
covid_data['daily_recovered'] = covid_data.groupby('country')['recovered'].diff()

In [26]:
# В таблице vaccinations_data достаточно будет преобразовать столбцы в формат datetime
vaccinations_data['date'] = pd.to_datetime(vaccinations_data['date'])

### Задания

In [27]:
# За какой период представлены данные в таблице covid_data?
covid_data['date'].sort_values()

4056    2020-01-22
4055    2020-01-22
4054    2020-01-22
4061    2020-01-22
4060    2020-01-22
           ...    
45506   2021-05-29
45490   2021-05-29
45433   2021-05-29
45523   2021-05-29
45437   2021-05-29
Name: date, Length: 86785, dtype: datetime64[ns]

In [28]:
# За какой период представлены данные в таблице vaccinations_data?
vaccinations_data['date'].sort_values()

28893   2020-12-02
28894   2020-12-03
21214   2020-12-04
28895   2020-12-04
21215   2020-12-05
           ...    
29639   2021-09-06
6809    2021-09-06
30024   2021-09-06
36494   2021-09-06
41328   2021-09-06
Name: date, Length: 42795, dtype: datetime64[ns]

In [29]:
# в результирующую таблицу должны попасть
# все записи из таблицы covid_data и из её пересечения с vaccinations_data
# но не более
covid_df = covid_data.merge(vaccinations_data, how= 'left', on= ['date', 'country'])
print(covid_df.shape)

(86785, 16)


In [30]:
# death_rate — общий процент смертей среди зафиксированных случаев (летальность)
covid_df['death_rate'] = (covid_df['deaths'] / covid_df['confirmed'])*100

In [31]:
# recover_rate — общий процент случаев выздоровления
covid_df['recover_rate'] = (covid_df['recovered'] / covid_df['confirmed'])*100

In [32]:
# Какова максимальная летальность в США (United States) за весь период?
covid_df[covid_df['country'] == 'United States'].max()

/tmp/ipykernel_20201/3186289417.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  covid_df[covid_df['country'] == 'United States'].max()


date                                   2021-05-29 00:00:00
country                                      United States
confirmed                                       33251939.0
deaths                                            594306.0
recovered                                        6399531.0
active                                          32657633.0
daily_confirmed                                   300310.0
daily_deaths                                        4561.0
daily_recovered                                   150267.0
total_vaccinations                             293705050.0
people_vaccinated                              167157043.0
people_vaccinated_per_hundred                         49.7
people_fully_vaccinated                        134418748.0
people_fully_vaccinated_per_hundred                  39.97
daily_vaccinations                               3384387.0
death_rate                                        7.189542
recover_rate                                     39.6987

In [33]:
# Чему равен средний процент выздоровевших в России (Russia)?
covid_df[covid_df['country'] == 'Russia']['recover_rate'].mean()

67.06273489477655

In [34]:
covid_df = pd.read_csv('data/covid_df.csv') 
covid_df['date'] = pd.to_datetime(covid_df['date'])
covid_df.head()

,Unnamed: 0,date,country,confirmed,deaths,recovered,active,daily_confirmed,daily_deaths,daily_recovered,total_vaccinations,people_vaccinated,people_vaccinated_per_hundred,people_fully_vaccinated,people_fully_vaccinated_per_hundred,daily_vaccinations,vaccines,death_rate,recover_rate
0,0,2020-02-24,Afghanistan,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
1,1,2020-02-25,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2,2,2020-02-26,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
3,3,2020-02-27,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
4,4,2020-02-28,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0


In [35]:
covid_df.to_csv('data/covid_df.csv')